## Review

Our last classsifier was very poor--it operated at chance--a coin flip would have had the same predictive power. A few things may have been going on that casued us to find no signal. I could be we down sampled our images too much and lost useful information, it could be that our model was poorly configured (it was), it could be we were using thr wrong model (we were), or it could be all of these. My gut (and eyes) say there is still plenty of information in our down sampled images, so we're going to try a more approproate model before we go back and try with larger images. 

We're going to build a convolutional neural network using keras (with Tensorflow as the backend). For starters, convolutional neural networks are much better at image classification than the mutli-layer perceptron used in the last post. 

In [1]:
# keeps warnings from printing for nicer blog output
import warnings 
warnings.filterwarnings('ignore') 

import os

import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Permute, Reshape, LSTM, Dropout, TimeDistributed, Dense, Activation, Flatten
from keras import optimizers

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')

Using TensorFlow backend.


In [2]:
os.chdir(os.path.expanduser('~/share/rkingdc-blog/regplot'))

We can see this is the same file we saw in the previous post, but with the image data as an added column. Each cell in that `img_series` column is a numpy ndarray of shape `(168,168)`. 

## Building the Model

We'll start by building our classifier. Code will be explained inline. 

In [3]:
input_dim1 = 256
lstm_size = 150
hidden_layer_size = 100
adam_parms = {'lr': 1e-4, 'beta_1': 0.9, 'beta_2': 0.999}

mod = Sequential()

mod.add(Permute((2,1,3), input_shape=(input_dim1,input_dim1,3)))
mod.add(Reshape(target_shape = (input_dim1,input_dim1*3)))

# our hidden layers
mod.add(LSTM(lstm_size, return_sequences=True))
mod.add(LSTM(lstm_size, return_sequences=True))

# dropout 
mod.add(Dropout(0.5))

mod.add(TimeDistributed(Dense(hidden_layer_size), input_shape=(input_dim1, lstm_size) ))

mod.add(Flatten())

mod.add(Dense(1, activation='sigmoid'))

mod.compile(optimizer=optimizers.Adam(**adam_parms), loss='binary_crossentropy', metrics=['accuracy'])
mod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 256, 256, 3)       0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 256, 768)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256, 150)          551400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256, 150)          180600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 150)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 256, 100)          15100     
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
__________

## Pre-procesing Data

In image processing we'll want to pre-proccess our images before we train a model on them, by adding some random stretching, blurring, rotating, etc. Keras has utilities included to make this easier. 

In [4]:
train_gen = ImageDataGenerator(rescale = 1/255)
test_gen = ImageDataGenerator(rescale = 1/255)

In [5]:
train = train_gen.flow_from_directory('data/imgs/train',
                                      shuffle=True,
                                      batch_size=36,
                                      class_mode='binary')
test = test_gen.flow_from_directory('data/imgs/test',
                                    shuffle=True,
                                    batch_size=36,
                                    class_mode='binary')

Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [6]:
mod.fit_generator(train,
       epochs=10,
       verbose=0,
       validation_data=test,
       callbacks=[csv_logger])

In [7]:
from datetime import date
mod.save(f'trained_model_1_{str(date.today())}.h5')

In [8]:
model_eval = mod.evaluate_generator(test, use_multiprocessing=True, workers=2)
print(mod.metrics_names)
print(model_eval)

['loss', 'acc']
[0.02431063937046565, 0.9917499966025353]
